# Setup

In [ ]:
# Docker and Milvus Standalone

# 1. wget https://github.com/milvus-io/milvus/releases/download/v2.0.2/milvus-standalone-docker-compose.yml -O docker-compose.yml
# 2. https://docs.docker.com/compose/install/
# 3. Start: sudo docker-compose up -d 
#     Check:sudo docker-compose ps
#     Stop: sudo docker-compose down
#     Delete data after stop: sudo rm -rf  volumes


In [2]:
# Python 3.7.1 or later is required

In [ ]:
# pymilvus

# python3 -m pip install pymilvus==2.0.2
#  Verify: python3 -c "from pymilvus import Collection"

# Import Libraries

In [ ]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)
import time
import numpy as np

# Read Embeddings

In [ ]:
X_train_features = np.loadtxt(os.path.join("0004",'X_train_features.txt'))
X_test_features = np.loadtxt(os.path.join("0004",'X_test_features.txt'))

In [ ]:
num_entities, dim = X_train_features.shape

In [ ]:
y_train = np.loadtxt(os.path.join("0004",'y_train.txt'))
y_test = np.loadtxt(os.path.join("0004",'y_test.txt'))

In [ ]:
train_images = np.loadtxt(os.path.join("0004",'train_images.txt'))
test_images = np.loadtxt(os.path.join("0004",'test_images.txt'))

# Connect to Server

In [ ]:
connections.connect("default", host="localhost", port="19530")

# Create Collection

In [ ]:
has = utility.has_collection("facenet")
print(f"Does collection face_net exist in Milvus: {has}")

In [ ]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64,
                is_primary=True, auto_id=False, description="primary id"),
    FieldSchema(name="label", dtype=DataType.INT64, description="label"),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR,
                dim=dim, description="vector")
]

schema = CollectionSchema(fields, "Embeddings for facenet")
print(fmt.format("Create collection `embed`"))
embed = Collection("embed", schema,
                          using='default', shards_num=2)

In [ ]:
print(fmt.format("Start inserting entities"))

entities = [
    [i for i in range(0, num_entities)],  # field pk
    y_train.reshape(-1,1),
    X_train_features,  # field embeddings
]
insert_result = embed.insert(entities)

In [ ]:
print(f"Number of Entities: {embed.num_entities}")

# Build Index on Entities

In [ ]:
import math

print(fmt.format("Start Creating index IVF_FLAT"))

index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 4 * math.sqrt(num_entities)},
}
hello_milvus.create_index("embeddings", index)

# Load to Memory

In [ ]:
print(fmt.format("Start loading"))

hello_milvus.load()

# Vector Similarity Search

## Similartiy Search

In [ ]:
search_params = {
    "metric_type": "l2",
    "params": {"nprobe": 330},
}

In [ ]:
result = hello_milvus.search(X_test_features, anns_field="embeddings", search_params, limit=3, output_fields=["label"])

# Result 

In [ ]:
result

# Realse Collection

In [ ]:
collection.release()

In [ ]:
# python3 hello_milvus.py